In [20]:
import torch
from torch import nn
from torch.nn import functional as F
from torch.utils.data import Dataset,DataLoader
from torchvision import transforms
import torchvision
import pandas as pd
import os
import re
from PIL import Image

In [26]:
class CIFAR10Dataset(Dataset):
    def __init__(self, file_path=[], crop_size_img=None, crop_size_label=None):
        """para:
            file_path(list): 数据和标签路径,列表元素第一个为图片路径，第二个为标签路径
        """
        # 1 正确读入图片和标签路径
        if len(file_path) != 2:
            raise ValueError("同时需要图片和标签文件夹的路径，图片路径在前")
        self.img_path = file_path[0]
        self.label_path = file_path[1]
        # 2 从路径中取出图片和标签数据的文件名保持到两个列表当中（程序中的数据来源）
        self.imgs = self.read_file(self.img_path)
        labels=pd.read_csv(file_path[1])
        self.labels = labels['y'].to_list()

        


    def __getitem__(self, index):
        
        # 从文件名中读取数据（图片和标签都是png格式的图像数据）
        img = self.imgs[index]
        img = Image.open(img)
        label = self.labels[index]
        img = self.img_transform(img, label)
        return img,label

    def __len__(self):
        return len(self.imgs)

    def read_file(self, path):
        """从文件夹中读取数据"""
        files_list = os.listdir(path)
        
        file_path_list = {int(re.search(r'(?<=train\\)[0-9]+',os.path.join(path, img)).group(0)):os.path.join(path, img) for img in files_list}
        return file_path_list

    def img_transform(self, img, label):
        """对图片和标签做一些数值处理"""
        transform = transforms.Compose(
            [
                transforms.ToTensor(),
                # transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
            ]
        )
        img = transform(img)

        return img


In [27]:
tensor_dataset=CIFAR10Dataset([r'D:\书籍资料整理\kaggle\cifar-10\train',r'D:\书籍资料整理\kaggle\cifar-10\label.csv'])

In [28]:
tensor_dataloader = DataLoader(tensor_dataset,   # 封装的对象
                               batch_size=2,     # 输出的batchsize
                               shuffle=True,     # 随机输出
                               num_workers=0)    # 只有1个进程

tensor([[[[0.6784, 0.6667, 0.6627,  ..., 0.6627, 0.6627, 0.6549],
          [0.6588, 0.6471, 0.6431,  ..., 0.6353, 0.6353, 0.6353],
          [0.6431, 0.6353, 0.6353,  ..., 0.6275, 0.6275, 0.6275],
          ...,
          [0.9333, 0.9176, 0.9176,  ..., 0.9137, 0.9137, 0.9216],
          [0.9333, 0.9098, 0.9137,  ..., 0.9098, 0.9176, 0.9176],
          [0.9294, 0.9098, 0.9059,  ..., 0.9059, 0.9176, 0.9176]],

         [[0.7020, 0.6902, 0.6863,  ..., 0.6863, 0.6863, 0.6784],
          [0.6824, 0.6667, 0.6667,  ..., 0.6588, 0.6588, 0.6588],
          [0.6667, 0.6588, 0.6588,  ..., 0.6510, 0.6510, 0.6510],
          ...,
          [0.8588, 0.8431, 0.8471,  ..., 0.8314, 0.8314, 0.8392],
          [0.8549, 0.8353, 0.8392,  ..., 0.8275, 0.8353, 0.8353],
          [0.8510, 0.8353, 0.8314,  ..., 0.8235, 0.8353, 0.8353]],

         [[0.7569, 0.7451, 0.7412,  ..., 0.7412, 0.7412, 0.7373],
          [0.7373, 0.7216, 0.7216,  ..., 0.7137, 0.7137, 0.7137],
          [0.7255, 0.7137, 0.7137,  ..., 0

In [25]:
def load_data_fashion_mnist(batch_size, resize=None):  # @save
    """下载Fashion-MNIST数据集，然后将其加载到内存中"""
    trans = [transforms.ToTensor()]
    if resize:
        trans.insert(0, transforms.Resize(resize))
    trans = transforms.Compose(trans)
    mnist_train = torchvision.datasets.FashionMNIST(
        root="D:/书籍资料整理/torch_data", train=True, transform=trans, download=True)
    print(mnist_train[2][0].shape)
    mnist_test = torchvision.datasets.FashionMNIST(
        root="D:/书籍资料整理/torch_data", train=False, transform=trans, download=True)
    
    print(mnist_test[1])
    return (data.DataLoader(mnist_train, batch_size, shuffle=True,
                            num_workers=2),
            data.DataLoader(mnist_test, batch_size, shuffle=False,
                            num_workers=2))
lr, num_epochs, batch_size = 0.05, 10, 256
train_iter, test_iter = load_data_fashion_mnist(batch_size, resize=96)

torch.Size([1, 96, 96])
(tensor([[[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]]]), 2)


AttributeError: 'Tensor' object has no attribute 'DataLoader'

In [ ]:
#使用这种方式,需要将图片按照类建文件夹并且放进去
#如
# root/dog/xxx.png
# root/dog/xxy.png
# root/dog/xxz.png

# root/cat/123.png
# root/cat/nsdf3.png
# root/cat/asd932_.png
data_transfrom = transforms.Compose([  # 对读取的图片进行以下指定操作
    transforms.Resize((32, 32)),     # 图片放缩为 (300, 300), 统一处理的图像最好设置为统一的大小,这样之后的处理也更加方便
    transforms.ToTensor(),             # 向量化,向量化时 每个点的像素值会除以255,整个向量中的元素值都在0-1之间      
])
 
img = datasets.ImageFolder('D:/书籍资料整理/kaggle/cifar-10/train', transform=data_transfrom)  # 指明读取的文件夹和读取方式,注意指明的是到文件夹的路径,不是到图片的路径
 
imgLoader = torch.utils.data.DataLoader(img, batch_size=2, shuffle=False, num_workers=1)  # 指定读取配置信息

